In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys  
import pandas as pd
import datetime
import pyodbc 
server = r'pdc-lst-str-02.corp.suek.ru'
#server = r'pdc-lst-str-02'
#server = r'10.92.6.165'
database = r'ces_tgk12' 
#username = r'sibgenco.local\dmitrievmm'
#username = r'SUEKCORP\dmitrievmm'
#password = r'gfhjkmlkzFylhtz10' 

username = r'SUEKCORP\omima'
password = r'VxjNOJV]!3'

conn = pyodbc.connect('DRIVER={FreeTDS};SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password+';TDS_VERSION=8.0;')
#conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';PORT=1433;DATABASE='+database+'')


import datetime
now = datetime.datetime.now() - datetime.timedelta(days=6)

In [8]:
import datetime
dataD10=str(datetime.date.today() - datetime.timedelta(days=15) - datetime.timedelta(days=6))
data_now=str(datetime.date.today() - datetime.timedelta(days=6))
dataD10

'2022-04-08'

In [3]:
# stmt = "SELECT * FROM INFORMATION_SCHEMA.TABLES ;"
# df = pd.read_sql(stmt,conn)
# df

In [4]:
# stmt = "SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'SellNorem';"
# df = pd.read_sql(stmt,conn)
# df

In [5]:
# cursor = conn.cursor()  
# qerty='SELECT TOP 10 * \
#         FROM ComDis.PeakHour'  
# GtpPlanValue = pd.read_sql(qerty,conn)
# GtpPlanValue

In [6]:
# cursor = conn.cursor()  
# qerty='SELECT TOP 1 * \
#         FROM ComDis.GtpPlanConsume'  
# GtpPlanValue = pd.read_sql(qerty,conn)
# GtpPlanValue

# РСВ

In [25]:
import time
start_time = time.time()

#Забираю 2 ценовые зонв одним запросом
cursor = conn.cursor()  
qerty='SELECT  SUM(Value) AS ValueSum, RegionName,PriceZone,Date \
        FROM ComDis.GtpPlanValue WHERE Date>=convert (datetime,\''+dataD10+' 0:00:00\') AND Date<convert (datetime,\''+data_now+' 0:00:00\') \
        GROUP BY PriceZone, RegionName, Date \
        ORDER BY RegionName,Date'  
GtpPlanValue = pd.read_sql(qerty,conn)
GtpPlanValue
print("РСВ--- %s seconds ---" % (time.time() - start_time))

РСВ--- 38.23733615875244 seconds ---


In [26]:
GtpPlanValue0=GtpPlanValue.copy()

In [27]:
GtpPlanValue[GtpPlanValue['RegionName'] == 'Ленинградская область']

,ValueSum,RegionName,PriceZone,Date
7920,5366.317,Ленинградская область,Европа,2022-04-08 00:00:00
7921,5154.095,Ленинградская область,Европа,2022-04-08 01:00:00
7922,5052.212,Ленинградская область,Европа,2022-04-08 02:00:00
7923,5004.448,Ленинградская область,Европа,2022-04-08 03:00:00
7924,5001.246,Ленинградская область,Европа,2022-04-08 04:00:00
...,...,...,...,...
8275,5546.808,Ленинградская область,Европа,2022-04-22 19:00:00
8276,5626.852,Ленинградская область,Европа,2022-04-22 20:00:00
8277,5691.159,Ленинградская область,Европа,2022-04-22 21:00:00
8278,5516.922,Ленинградская область,Европа,2022-04-22 22:00:00


In [28]:
GtpPlanValue=GtpPlanValue0.copy()
GtpPlanValue['hour']=GtpPlanValue['Date'].dt.hour.astype(int)
#GtpPlanValue['hour']=str(GtpPlanValue['hour']-([1]*len(GtpPlanValue)))
GtpPlanValue['Названия строк']=GtpPlanValue['Date'].dt.year
GtpPlanValue['N месяца']=GtpPlanValue['Date'].dt.month
GtpPlanValue['N дня']=GtpPlanValue['Date'].dt.day
GtpPlanValue
for i in range(24):
    GtpPlanValue[str(i)+'-'+str(i+1)]=0
for i in range(24):
    GtpPlanValue[str(i)+'-'+str(i+1)][GtpPlanValue['hour']==i]=GtpPlanValue['ValueSum'][GtpPlanValue['hour']==i]
    
    
GtpPlanValue=GtpPlanValue.groupby(['RegionName','PriceZone','Названия строк','N месяца','N дня']).max()
GtpPlanValue=GtpPlanValue.reset_index()
GtpPlanValue=GtpPlanValue[['RegionName','PriceZone','Названия строк','N месяца','N дня']+[str(c)+'-'+str(c+1)for c in range(24)]]
GtpPlanValue

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

,RegionName,PriceZone,Названия строк,N месяца,N дня,0-1,1-2,2-3,3-4,4-5,...,14-15,15-16,16-17,17-18,18-19,19-20,20-21,21-22,22-23,23-24
0,Алтайский край,Сибирь,2022,4,8,929.215,956.335,1052.300,1167.376,1259.906,...,1240.367,1231.429,1237.430,1217.284,1135.093,1038.926,969.623,930.670,920.447,918.435
1,Алтайский край,Сибирь,2022,4,9,917.284,932.177,977.625,1050.135,1148.468,...,1215.019,1213.058,1215.250,1178.957,1106.903,1024.462,964.519,927.771,912.385,909.680
2,Алтайский край,Сибирь,2022,4,10,908.696,920.293,957.581,1011.027,1104.373,...,1212.546,1217.158,1218.712,1186.478,1107.466,1022.991,959.047,917.224,910.708,905.605
3,Алтайский край,Сибирь,2022,4,11,918.191,951.595,1046.355,1168.105,1254.384,...,1223.407,1219.516,1238.824,1224.006,1135.526,1037.853,967.639,927.688,919.510,917.268
4,Алтайский край,Сибирь,2022,4,12,926.248,956.458,1054.180,1172.231,1267.699,...,1229.595,1221.402,1230.389,1222.825,1141.726,1043.083,976.051,936.808,924.284,917.659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,город Севастополь,Европа,2022,4,18,172.583,161.554,154.666,151.661,151.622,...,209.708,208.770,209.821,210.740,217.109,223.766,229.822,228.087,212.689,194.186
1031,город Севастополь,Европа,2022,4,19,172.473,160.615,152.700,150.815,151.756,...,217.094,214.148,214.250,216.130,220.015,229.770,233.753,230.040,217.028,195.057
1032,город Севастополь,Европа,2022,4,20,176.802,161.209,153.134,151.934,153.739,...,222.140,223.057,223.025,220.964,220.734,233.311,238.210,234.090,216.785,196.634
1033,город Севастополь,Европа,2022,4,21,175.698,162.415,153.870,151.869,152.770,...,214.833,214.900,213.930,215.973,220.807,229.711,233.685,230.067,212.470,192.847


In [29]:
GtpPlanValue_1=GtpPlanValue[GtpPlanValue['PriceZone']=='Европа'].drop((['PriceZone']), axis=1)
GtpPlanValue_2=GtpPlanValue[GtpPlanValue['PriceZone']=='Сибирь'].drop((['PriceZone']), axis=1)


In [30]:
GtpPlanValue_1_mi=pd.concat(([GtpPlanValue_1[GtpPlanValue_1['RegionName']==reg].reset_index(drop=True)   for reg in GtpPlanValue_1['RegionName'].unique()]), axis=1, keys=[reg for reg in GtpPlanValue_1['RegionName'].unique()])

GtpPlanValue_1_mi[('Unnamed: 0_level_0','Названия строк')]=GtpPlanValue_1_mi[('Астраханская область','Названия строк')]
GtpPlanValue_1_mi[('Unnamed: 1_level_0','N месяца')]=GtpPlanValue_1_mi[('Астраханская область','N месяца')]
GtpPlanValue_1_mi[('Unnamed: 2_level_0','N дня')]=GtpPlanValue_1_mi[('Астраханская область','N дня')]

for col in GtpPlanValue_1['RegionName'].unique():
    GtpPlanValue_1_mi=GtpPlanValue_1_mi.drop([(col,'Названия строк'),(col,'N месяца'),(col,'N дня'),(col,'RegionName')], axis=1)
GtpPlanValue_1_mi.to_pickle('../models/data_RSV_pred_4_0-1cz.pkl')




print(GtpPlanValue_1_mi)

   Астраханская область                                                        \
                    0-1      1-2      2-3      3-4      4-5      5-6      6-7   
0               376.980  370.875  368.655  370.959  382.375  405.494  427.904   
1               371.612  363.412  360.188  360.553  365.734  373.352  386.454   
2               362.578  353.571  348.715  348.879  354.479  363.344  376.112   
3               352.697  346.344  344.436  346.837  358.775  378.844  402.251   
4               344.086  336.630  334.710  338.250  349.421  369.236  395.129   
5               361.652  355.967  354.613  357.178  367.857  388.635  411.964   
6               361.923  357.613  355.516  359.017  368.766  389.121  410.918   
7               360.800  355.492  353.230  356.699  366.194  388.041  408.537   
8               372.070  364.694  361.706  362.042  367.368  373.645  390.394   
9               363.224  353.690  351.031  351.078  353.292  361.459  378.446   
10              363.701  355

In [31]:
GtpPlanValue_2_mi=pd.concat(([GtpPlanValue_2[GtpPlanValue_2['RegionName']==reg].reset_index(drop=True)   for reg in GtpPlanValue_2['RegionName'].unique()]), axis=1, keys=[reg for reg in GtpPlanValue_2['RegionName'].unique()])

GtpPlanValue_2_mi[('Unnamed: 0_level_0','Названия строк')]=GtpPlanValue_2_mi[('Томская область','Названия строк')]
GtpPlanValue_2_mi[('Unnamed: 1_level_0','N месяца')]=GtpPlanValue_2_mi[('Томская область','N месяца')]
GtpPlanValue_2_mi[('Unnamed: 2_level_0','N дня')]=GtpPlanValue_2_mi[('Томская область','N дня')]

for col in GtpPlanValue_2['RegionName'].unique():
    GtpPlanValue_2_mi=GtpPlanValue_2_mi.drop([(col,'Названия строк'),(col,'N месяца'),(col,'N дня'),(col,'RegionName')], axis=1)
GtpPlanValue_2_mi.to_pickle('../models/data_RSV_pred_4_0-2cz.pkl')


print(GtpPlanValue_2_mi)

   Алтайский край                                                             \
              0-1      1-2       2-3       3-4       4-5       5-6       6-7   
0         929.215  956.335  1052.300  1167.376  1259.906  1303.257  1293.922   
1         917.284  932.177   977.625  1050.135  1148.468  1224.535  1246.745   
2         908.696  920.293   957.581  1011.027  1104.373  1193.128  1233.136   
3         918.191  951.595  1046.355  1168.105  1254.384  1302.478  1295.695   
4         926.248  956.458  1054.180  1172.231  1267.699  1311.812  1303.365   
5         920.945  950.885  1050.745  1167.413  1260.075  1301.194  1295.294   
6         919.597  948.742  1045.764  1164.456  1256.921  1289.339  1282.805   
7         898.143  925.104  1015.526  1139.650  1224.399  1261.830  1256.786   
8         889.838  906.152   955.057  1038.934  1137.823  1189.581  1205.845   
9         889.939  894.113   944.370   996.733  1094.303  1165.868  1187.988   
10        868.188  897.139   990.745  11

In [32]:
GtpPlanValue_2_mi

Алтайский край                                                             \
              0-1      1-2       2-3       3-4       4-5       5-6       6-7   
0         929.215  956.335  1052.300  1167.376  1259.906  1303.257  1293.922   
1         917.284  932.177   977.625  1050.135  1148.468  1224.535  1246.745   
2         908.696  920.293   957.581  1011.027  1104.373  1193.128  1233.136   
3         918.191  951.595  1046.355  1168.105  1254.384  1302.478  1295.695   
4         926.248  956.458  1054.180  1172.231  1267.699  1311.812  1303.365   
5         920.945  950.885  1050.745  1167.413  1260.075  1301.194  1295.294   
6         919.597  948.742  1045.764  1164.456  1256.921  1289.339  1282.805   
7         898.143  925.104  1015.526  1139.650  1224.399  1261.830  1256.786   
8         889.838  906.152   955.057  1038.934  1137.823  1189.581  1205.845   
9         889.939  894.113   944.370   996.733  1094.303  1165.868  1187.988   
10        868.188  897.139   990.745  1109.315  1203.669  1252.014  1242.905   
11        869.310  892.588   985.458  1101.207  1192.790  1241.763  1235.371   
12        881.024  910.856  1007.247  1135.457  1227.851  1280.374  1277.694   
13        895.573  932.264  1028.737  1145.709  1241.946  1287.627  1288.465   
14        912.325  939.174  1030.728  1154.617  1252.002  1297.446  1297.564   

                                  ... Томская область                    \
         7-8       8-9      9-10  ...           17-18    18-19    19-20   
0   1271.625  1253.415  1248.372  ...         848.413  826.378  792.575   
1   1242.313  1232.777  1224.022  ...         843.780  816.203  784.677   
2   1238.510  1228.459  1216.312  ...         856.538  826.076  789.324   
3   1277.862  1248.299  1241.852  ...         876.421  842.804  804.000   
4   1278.450  1264.579  1251.654  ...         869.552  837.479  799.103   
5   1276.554  1256.910  1248.982  ...         868.612  835.546  794.820   
6   1255.623  1239.274  1232.776  ...         839.961  809.517  766.882   
7   1223.393  1211.006  1204.555  ...         848.403  816.816  780.919   
8   1184.430  1172.683  1158.279  ...         811.856  785.962  757.061   
9   1170.345  1174.797  1157.606  ...         820.082  794.943  758.146   
10  1211.251  1200.956  1198.159  ...         839.296  811.049  771.986   
11  1212.346  1204.623  1196.330  ...         843.390  815.245  775.914   
12  1261.525  1247.102  1243.200  ...         818.828  789.408  749.951   
13  1275.842  1268.882  1257.849  ...         835.210  803.666  764.497   
14  1281.538  1270.147  1261.502  ...         857.075  826.113  786.450   

                                       Unnamed: 0_level_0 Unnamed: 1_level_0  \
      20-21    21-22    22-23    23-24     Названия строк           N месяца   
0   762.654  745.441  734.017  717.264               2022                  4   
1   760.445  742.369  729.985  726.700               2022                  4   
2   760.225  744.529  732.687  726.996               2022                  4   
3   771.518  753.758  742.004  736.600               2022                  4   
4   767.864  751.504  741.058  735.739               2022                  4   
5   765.006  747.819  739.035  733.356               2022                  4   
6   735.119  722.199  711.175  705.795               2022                  4   
7   749.834  735.078  721.667  715.359               2022                  4   
8   731.473  713.542  701.152  693.807               2022                  4   
9   727.490  709.398  697.900  691.549               2022                  4   
10  738.116  719.403  707.746  701.856               2022                  4   
11  727.195  708.346  696.786  690.640               2022                  4   
12  717.032  698.717  688.070  682.036               2022                  4   
13  730.341  718.556  707.768  701.376               2022                  4   
14  752.218  737.350  725.740  718.741               2022                  4   

   Un

# ВСВГО

In [33]:
import time
start_time = time.time()

#Забираю 2 ценовые зонв одним запросом
cursor = conn.cursor()  
qerty='SELECT  Value AS ValueSum, RegionName,PriceZone,Date \
        FROM ComDis.RegionConsumeForecast WHERE Date>=convert (datetime,\''+dataD10+' 0:00:00\') AND Date<convert (datetime,\''+data_now+' 0:00:00\') \
        ORDER BY RegionName,Date'  
RegionConsumeForecast = pd.read_sql(qerty,conn)
RegionConsumeForecast
print("ВСВГО--- %s seconds ---" % (time.time() - start_time))

ВСВГО--- 1.679152488708496 seconds ---


In [34]:
# RegionConsumeForecast[RegionConsumeForecast["RegionName"]=='Ленинградская область']

In [35]:
RegionConsumeForecast['hour']=RegionConsumeForecast['Date'].dt.hour.astype(int)
RegionConsumeForecast['Названия строк']=RegionConsumeForecast['Date'].dt.year
RegionConsumeForecast['N месяца']=RegionConsumeForecast['Date'].dt.month
RegionConsumeForecast['N дня']=RegionConsumeForecast['Date'].dt.day
RegionConsumeForecast
for i in range(24):
    RegionConsumeForecast[str(i)+'-'+str(i+1)]=0
for i in range(24):
    RegionConsumeForecast[str(i)+'-'+str(i+1)][RegionConsumeForecast['hour']==i]=RegionConsumeForecast['ValueSum'][RegionConsumeForecast['hour']==i]
    
    
RegionConsumeForecast=RegionConsumeForecast.groupby(['RegionName','PriceZone','Названия строк','N месяца','N дня']).max()
RegionConsumeForecast=RegionConsumeForecast.reset_index()
RegionConsumeForecast=RegionConsumeForecast[['RegionName','PriceZone','Названия строк','N месяца','N дня']+[str(c)+'-'+str(c+1)for c in range(24)]]
RegionConsumeForecast

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dis

,RegionName,PriceZone,Названия строк,N месяца,N дня,0-1,1-2,2-3,3-4,4-5,...,14-15,15-16,16-17,17-18,18-19,19-20,20-21,21-22,22-23,23-24
0,Алтайский край,Сибирь,2022,4,8,989.777112,1057.353643,1197.922920,1324.277523,1409.762024,...,1327.596953,1349.898809,1364.512110,1276.897614,1175.067002,1085.792216,1031.992703,998.476104,979.226567,983.480671
1,Алтайский край,Сибирь,2022,4,9,968.281126,1026.860777,1098.654048,1202.236621,1303.389761,...,1309.931257,1312.835807,1354.519871,1275.332161,1161.864954,1084.728539,1024.286952,992.212144,961.744324,965.571458
2,Алтайский край,Сибирь,2022,4,10,971.254065,1009.271703,1064.494915,1146.061523,1246.787061,...,1287.930737,1304.235746,1350.774026,1289.910823,1177.866980,1078.537441,1015.030113,974.335286,949.890118,939.877342
3,Алтайский край,Сибирь,2022,4,11,994.159013,1072.161990,1204.166533,1331.248320,1421.005844,...,1353.474940,1377.929356,1407.432866,1317.340360,1204.845671,1098.577937,1058.686893,1008.732857,987.157246,989.608068
4,Алтайский край,Сибирь,2022,4,12,988.664721,1062.858629,1194.673082,1326.820550,1394.626278,...,1332.059879,1360.496615,1391.692041,1312.560771,1199.793550,1106.458747,1049.098421,992.510767,991.646206,997.379966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,Ярославская область,Европа,2022,4,18,831.781665,818.314196,823.125665,817.126057,828.087847,...,1093.257299,1078.090280,1048.731924,1057.389045,1061.336519,1076.503820,1048.920669,991.623598,924.072465,882.556391
1076,Ярославская область,Европа,2022,4,19,873.830006,855.054593,842.840688,851.234718,853.156422,...,1083.381998,1072.024256,1053.922259,1049.786983,1053.310184,1075.498851,1055.016610,1003.702182,928.934573,897.978911
1077,Ярославская область,Европа,2022,4,20,878.979301,857.859174,854.044081,861.332518,869.168788,...,1061.167228,1049.439470,1038.825684,1028.025511,1036.794977,1042.844917,1029.038393,985.853111,921.150889,892.416474
1078,Ярославская область,Европа,2022,4,21,857.816992,843.560203,839.625984,835.675382,844.205235,...,1047.821076,1033.245972,1018.278902,1009.013930,1029.887837,1042.030130,1026.795321,988.178270,917.067577,870.152566


In [36]:
RegionConsumeForecast_1=RegionConsumeForecast[RegionConsumeForecast['PriceZone']=='Европа'].drop((['PriceZone']), axis=1)
RegionConsumeForecast_2=RegionConsumeForecast[RegionConsumeForecast['PriceZone']=='Сибирь'].drop((['PriceZone']), axis=1)


In [37]:
RegionConsumeForecast_1_mi=pd.concat(([RegionConsumeForecast_1[RegionConsumeForecast_1['RegionName']==reg].reset_index(drop=True)   for reg in RegionConsumeForecast_1['RegionName'].unique()]), axis=1, keys=[reg for reg in RegionConsumeForecast_1['RegionName'].unique()])

RegionConsumeForecast_1_mi[('Unnamed: 0_level_0','Названия строк')]=RegionConsumeForecast_1_mi[('Астраханская область','Названия строк')]
RegionConsumeForecast_1_mi[('Unnamed: 1_level_0','N месяца')]=RegionConsumeForecast_1_mi[('Астраханская область','N месяца')]
RegionConsumeForecast_1_mi[('Unnamed: 2_level_0','N дня')]=RegionConsumeForecast_1_mi[('Астраханская область','N дня')]

for col in RegionConsumeForecast_1['RegionName'].unique():
    RegionConsumeForecast_1_mi=RegionConsumeForecast_1_mi.drop([(col,'Названия строк'),(col,'N месяца'),(col,'N дня'),(col,'RegionName')], axis=1)
RegionConsumeForecast_1_mi.to_pickle('../models/data_VSVGO_pred_4_0-1cz.pkl')

print(RegionConsumeForecast_1_mi)


   Астраханская область                                                  \
                    0-1         1-2         2-3         3-4         4-5   
0            360.695321  355.602798  356.613343  363.753368  384.054644   
1            352.517514  345.445150  345.442238  348.471737  356.543616   
2            354.530952  349.053241  345.685932  347.287151  351.129791   
3            352.671013  346.225252  346.520638  350.251184  366.227524   
4            342.248176  337.986713  336.057756  340.688718  353.518920   
5            325.082568  321.085063  320.051757  326.219789  344.858265   
6            333.305562  329.715423  327.844006  334.997799  354.858026   
7            350.049599  346.101609  342.153941  350.035368  365.709207   
8            354.037554  345.933702  343.876903  346.937377  354.079654   
9            347.228816  339.151763  336.640895  339.062545  343.188604   
10           330.636475  324.443094  322.374124  328.554254  342.970373   
11           350.378452  

In [38]:
RegionConsumeForecast_2_mi=pd.concat(([RegionConsumeForecast_2[RegionConsumeForecast_2['RegionName']==reg].reset_index(drop=True)   for reg in RegionConsumeForecast_2['RegionName'].unique()]), axis=1, keys=[reg for reg in RegionConsumeForecast_2['RegionName'].unique()])

RegionConsumeForecast_2_mi[('Unnamed: 0_level_0','Названия строк')]=RegionConsumeForecast_2_mi[('Томская область','Названия строк')]
RegionConsumeForecast_2_mi[('Unnamed: 1_level_0','N месяца')]=RegionConsumeForecast_2_mi[('Томская область','N месяца')]
RegionConsumeForecast_2_mi[('Unnamed: 2_level_0','N дня')]=RegionConsumeForecast_2_mi[('Томская область','N дня')]

for col in RegionConsumeForecast_2['RegionName'].unique():
    RegionConsumeForecast_2_mi=RegionConsumeForecast_2_mi.drop([(col,'Названия строк'),(col,'N месяца'),(col,'N дня'),(col,'RegionName')], axis=1)
RegionConsumeForecast_2_mi.to_pickle('../models/data_VSVGO_pred_4_0-2cz.pkl')


print(RegionConsumeForecast_2_mi)

   Алтайский край                                                      \
              0-1          1-2          2-3          3-4          4-5   
0      989.777112  1057.353643  1197.922920  1324.277523  1409.762024   
1      968.281126  1026.860777  1098.654048  1202.236621  1303.389761   
2      971.254065  1009.271703  1064.494915  1146.061523  1246.787061   
3      994.159013  1072.161990  1204.166533  1331.248320  1421.005844   
4      988.664721  1062.858629  1194.673082  1326.820550  1394.626278   
5      993.218645  1050.110481  1199.194472  1308.595848  1378.614063   
6      988.925643  1058.301158  1187.141278  1308.500920  1392.134833   
7      968.110123  1043.972182  1189.995354  1293.601112  1391.892677   
8      925.822067   968.987330  1034.582992  1144.186800  1236.074770   
9      912.993958   955.186521  1001.521109  1108.037597  1222.418609   
10     927.150251  1012.005111  1122.737681  1249.989279  1329.818465   
11     946.841458  1011.759264  1150.061146  1287.9

In [39]:
RegionConsumeForecast_1_mi

Астраханская область                                                  \
                    0-1         1-2         2-3         3-4         4-5   
0            360.695321  355.602798  356.613343  363.753368  384.054644   
1            352.517514  345.445150  345.442238  348.471737  356.543616   
2            354.530952  349.053241  345.685932  347.287151  351.129791   
3            352.671013  346.225252  346.520638  350.251184  366.227524   
4            342.248176  337.986713  336.057756  340.688718  353.518920   
5            325.082568  321.085063  320.051757  326.219789  344.858265   
6            333.305562  329.715423  327.844006  334.997799  354.858026   
7            350.049599  346.101609  342.153941  350.035368  365.709207   
8            354.037554  345.933702  343.876903  346.937377  354.079654   
9            347.228816  339.151763  336.640895  339.062545  343.188604   
10           330.636475  324.443094  322.374124  328.554254  342.970373   
11           350.378452  343.677785  341.800768  339.538062  354.187136   
12           349.210565  344.048799  342.200221  347.223783  361.514950   
13           350.138146  350.726305  339.702227  345.400305  358.477258   
14           357.258163  353.243105  348.002263  353.232929  367.817970   

                                                                ...  \
           5-6         6-7         7-8         8-9        9-10  ...   
0   408.472128  424.008048  439.791791  440.700891  438.813535  ...   
1   369.702556  389.698554  410.449904  418.355623  423.320523  ...   
2   360.526336  373.242792  389.945723  397.188924  400.395236  ...   
3   388.739380  400.021337  417.099915  425.447907  423.453287  ...   
4   378.509645  392.636777  413.284460  418.608658  417.269483  ...   
5   367.881167  382.838570  401.105579  404.096768  401.877755  ...   
6   391.587528  401.248631  422.986779  430.046335  426.675111  ...   
7   393.227970  414.284382  433.528236  442.205131  443.688197  ...   
8   371.284799  390.496543  409.235642  417.699317  421.842789  ...   
9   363.301081  381.637857  401.911759  409.054037  409.176795  ...   
10  373.793088  386.258816  404.714758  409.395559  412.562320  ...   
11  380.012242  391.225385  408.672197  411.688601  407.402373  ...   
12  391.273746  402.757114  418.446438  426.612932  427.411893  ...   
13  388.637375  406.904867  428.505635  433.780483  439.223949  ...   
14  397.647305  420.115452  442.202759  446.071416  447.881354  ...   

   Ярославская область                                                      \
                 17-18        18-19        19-20        20-21        21-22   
0          1024.379470  1027.830783  1049.270659  1035.864700   991.561747   
1           947.833428   959.485443   975.882944   966.542360   931.262523   
2           947.317902   946.537521   969.586256   986.772611   918.566898   
3          1013.637271  1017.893878  1029.098690  1016.642262   955.339409   
4          1038.320580  1055.968703  1051.037742  1027.444150   978.756942   
5          1051.975008  1047.596991  1059.894042  1043.352752   991.444207   
6          1018.289044  1016.601319  1035.615109  1027.043817   973.695899   
7          1009.808441  1015.541579  1032.839678  1024.879656   983.236483   
8           972.150592   979.150495   991.312897   985.755257   948.602557   
9           953.460383   962.484349   978.485299   994.498071   957.184001   
10         1057.389045  1061.336519  1076.503820  1048.920669   991.623598   
11         1049.786983  1053.310184  1075.498851  1055.016610  1003.702182   
12         1028.025511  1036.794977  1042.844917  1029.038393   985.853111   
13         1009.013930  1029.887837  1042.030130  1026.795321   988.178270   
14          961.506555   979.113068   995.901783  1005.126018   968.544301   

                           Unnamed: 0_level_0 Unnamed: 1_level_0  \
         22-23       23-24     Названия строк           N месяца   
0   926.701168  894.907379               2022           

In [40]:
RegionConsumeForecast_2_mi

Алтайский край                                                      \
              0-1          1-2          2-3          3-4          4-5   
0      989.777112  1057.353643  1197.922920  1324.277523  1409.762024   
1      968.281126  1026.860777  1098.654048  1202.236621  1303.389761   
2      971.254065  1009.271703  1064.494915  1146.061523  1246.787061   
3      994.159013  1072.161990  1204.166533  1331.248320  1421.005844   
4      988.664721  1062.858629  1194.673082  1326.820550  1394.626278   
5      993.218645  1050.110481  1199.194472  1308.595848  1378.614063   
6      988.925643  1058.301158  1187.141278  1308.500920  1392.134833   
7      968.110123  1043.972182  1189.995354  1293.601112  1391.892677   
8      925.822067   968.987330  1034.582992  1144.186800  1236.074770   
9      912.993958   955.186521  1001.521109  1108.037597  1222.418609   
10     927.150251  1012.005111  1122.737681  1249.989279  1329.818465   
11     946.841458  1011.759264  1150.061146  1287.981394  1354.693128   
12     942.829488  1025.027171  1164.670085  1306.486713  1362.618327   
13     965.729385  1072.938192  1192.292737  1317.678038  1415.487569   
14     978.675889  1064.093401  1204.148834  1339.698531  1430.907055   

                                                                     ...  \
            5-6          6-7          7-8          8-9         9-10  ...   
0   1416.809463  1417.375326  1388.045693  1388.906826  1361.449165  ...   
1   1341.541974  1344.781687  1329.637682  1319.463980  1321.170957  ...   
2   1306.207532  1325.299989  1308.388004  1312.009107  1302.090035  ...   
3   1437.622070  1443.823672  1435.398447  1392.232742  1378.944444  ...   
4   1411.862914  1398.103100  1378.614405  1351.425484  1333.071003  ...   
5   1396.268833  1378.880639  1350.121844  1325.108868  1314.092215  ...   
6   1415.160511  1391.988748  1353.134655  1335.825379  1314.040475  ...   
7   1397.962607  1365.696716  1320.865760  1305.481192  1296.195114  ...   
8   1293.379939  1290.419407  1265.844243  1251.822287  1244.225225  ...   
9   1265.960400  1269.701974  1258.467442  1244.060410  1227.548266  ...   
10  1366.578927  1358.477136  1330.265444  1322.475670  1312.071789  ...   
11  1356.490196  1340.781808  1318.704658  1309.042493  1297.440724  ...   
12  1388.430162  1375.626908  1347.288490  1325.420872  1307.543500  ...   
13  1425.936641  1425.769395  1387.403734  1363.590151  1343.929864  ...   
14  1441.826991  1427.702299  1390.406063  1374.732299  1366.512067  ...   

   Томская область                                                  \
             17-18       18-19       19-20       20-21       21-22   
0       979.241622  935.755824  889.099707  860.161785  845.702242   
1       967.273529  931.662806  890.766023  862.705452  847.159727   
2       969.878223  931.858434  890.073323  865.917976  850.121932   
3       984.324931  930.244461  889.018980  861.741414  845.683935   
4       981.214276  923.657622  875.859884  851.722487  841.052991   
5       985.348023  937.789265  884.663431  860.620611  845.881845   
6       983.247993  935.431170  885.989692  857.597191  843.236535   
7       974.859184  937.742676  895.371708  867.138110  850.015469   
8       968.604361  932.389941  894.659471  866.399743  848.771964   
9       946.980626  905.937843  874.993147  849.398627  833.773567   
10      964.518396  931.810221  889.930683  857.210785  843.926836   
11     1004.183587  950.393843  914.420892  880.928396  867.687687   
12      967.286456  916.093978  867.616330  841.806109  831.238030   
13      967.876780  921.277669  873.632468  846.772444  832.874306   
14      983.608116  940.776401  891.566051  863.029645  846.402117   

                           Unnamed: 0_level_0 Unnamed: 1_level_0  \
         22-23       23-24     Названия строк           N месяца   
0   838.760798  831.780475               2022                  4   
1   836.478519  831.751122               2022                  4   
2   8